# 1st Place Solution "Cassava Leaf Disease Classification"

This is the inference notebook of our final submission which scored ~91.3% on public and private leaderboard. We used an ensemble of four different models and stacked those models together using a mean approach.

You can find the according training code in these notebooks:

* [EfficientNet B4 (TPU Training)](https://www.kaggle.com/jannish/cassava-leaf-disease-efficientnetb4-tpu)
* [ResNext50_32x4d (GPU Training)](https://www.kaggle.com/hiarsl/cassava-leaf-disease-resnext50)
* [ViT (TPU Training)](https://www.kaggle.com/sebastiangnther/cassava-leaf-disease-vit-tpu-training)

In order to find the final combination of all the models we tested, we iteratively tried different ensembles using this notebook:

* [Ensembling by using OOF predictions](https://www.kaggle.com/jannish/cassava-leaf-disease-finding-final-ensembles)

Our final submission first averaged the probabilities of the predicted classes of ViT and ResNext. This averaged probability vector was then merged with the predicted probabilities of EfficientnetB4 and MobileNet(CropNet) in a second stage. For this purpose, the values were simply summed up.

Finally, we would like to thank all the Kagglers who posted their notebooks and gave valuable hints on which models to try!


In [2]:
!pip install --quiet tensorflow==2.9.2 albumentations timm tensorflow_hub torcheval

In [7]:
import sys
import os
import random
import json
import gc
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score
from functools import partial
from albumentations import (Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, CenterCrop, 
                            HorizontalFlip, VerticalFlip, Rotate, ShiftScaleRotate, Transpose)
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from tensorflow import keras

# sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
# sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, random_split
from torcheval.metrics.functional import multiclass_f1_score
from torchvision import transforms

In [8]:
path = "Data/"
image_path = path+"train_images/"

IMAGE_SIZE = (512,512)
submission_df = pd.DataFrame(columns={"image_id","label"})
ground_truth_df = pd.DataFrame(columns={"image_id","label"})

# Prepare data

In [9]:
class LeafDataset(Dataset):
    def __init__(self, root_path, transform=None):
        self.image_path = root_path + '/train_images'
        self.labels = pd.read_csv(root_path + '/train.csv')
        self.transform = transform

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_path, self.labels['image_id'][idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        if "label" in self.labels.columns:
            label = self.labels['label'][idx]
            img_id = self.labels['image_id'][idx]
            sample = (image, label, img_id)
        else:
            sample = (image)
        return sample

In [10]:
transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

total_dataset = LeafDataset('./Data', transform=transform)
subsets = random_split(total_dataset, [0.4, 0.2, 0.4], generator=torch.Generator().manual_seed(42))
test_dataloader = DataLoader(subsets[2], batch_size=1, shuffle=False, num_workers=1)

In [11]:
for image, label, img_id in test_dataloader:
    row = pd.DataFrame({"image_id": img_id, "label": 0}, index=[0])
    ground_truth_row = pd.DataFrame({"image_id": img_id, "label": label}, index=[0])
    submission_df = pd.concat([submission_df, row], ignore_index=True)
    ground_truth_df = pd.concat([ground_truth_df, ground_truth_row], ignore_index=True)

In [15]:
submission_df

,image_id,label
0,4257577206.jpg,0
1,3158652079.jpg,0
2,545843686.jpg,0
3,2828087857.jpg,0
4,3434116989.jpg,0
...,...,...
8553,812377482.jpg,0
8554,3610667873.jpg,0
8555,3309247762.jpg,0
8556,1307416806.jpg,0


In [16]:
ground_truth_df

,image_id,label
0,4257577206.jpg,0
1,3158652079.jpg,4
2,545843686.jpg,3
3,2828087857.jpg,3
4,3434116989.jpg,4
...,...,...
8553,812377482.jpg,2
8554,3610667873.jpg,3
8555,3309247762.jpg,3
8556,1307416806.jpg,1


# Used models in the final submission

In [17]:
# We used this flag to test combinations using only TF.Keras models
onlykeras = False
        
used_models_pytorch = {"vit2020": [f'cassava-leaf-disease-1st-place-models/vit/vit_base_patch16_384_fold_{fold}.h5' for fold in [0,1,2,3,4]],
                       "resnext": [f'cassava-leaf-disease-1st-place-models/resnext50_32x4d/resnext50_32x4d_fold{fold}_best.pth' for fold in [0,1,2,3,4]]}

used_models_keras = {"mobilenet": "cassava-leaf-disease-1st-place-models/cropnet_mobilenetv3/cropnet",
                     "efficientnetb4": "cassava-leaf-disease-1st-place-models/efficientnetb4/efficientnetb4_all_e14.h5"}

# We used this flag for testing different ensembling approaches
stacked_mean = True

# ResNext50_32x4d

In [18]:
class CustomResNext(nn.Module):
        def __init__(self, model_name='resnext50_32x4d', pretrained=False):
            super().__init__()
            self.model = timm.create_model(model_name, pretrained=pretrained)
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, 5)

        def forward(self, x):
            x = self.model(x)
            return x

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_path_id'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        image = cv2.imread(file_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

if "resnext" in used_models_pytorch:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def get_transforms():
        return Compose([Resize(512, 512),
                        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                        ToTensorV2()])

    def inference(model, states, test_loader, device):
        model.to(device)

        probabilities = []
        for i, (images) in enumerate(test_loader):
            images = images.to(device)
            avg_preds = []
            for state in states:
                model.load_state_dict(state['model'])
                model.eval()
                with torch.no_grad():
                    y_preds = model(images)
                avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probabilities.append(avg_preds)
        return np.concatenate(probabilities)
    

    predictions_resnext = pd.DataFrame(columns={"image_id"})
    predictions_resnext["image_id"] = submission_df["image_id"].values
    predictions_resnext['image_path_id'] = image_path + predictions_resnext['image_id'].astype(str)

    model = CustomResNext('resnext50_32x4d', pretrained=False)
    states = [torch.load(f) for f in used_models_pytorch["resnext"]]

    test_dataset = TestDataset(predictions_resnext, transform=get_transforms())
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
    predictions = inference(model, states, test_loader, device)

    predictions_resnext['resnext'] = [np.squeeze(p) for p in predictions]
    predictions_resnext = predictions_resnext.drop(["image_path_id"], axis=1)
    

    torch.cuda.empty_cache()
    try:
        del(model)
        del(states)
    except:
        pass
    gc.collect()

# ViT

In [19]:
if "vit2020" in used_models_pytorch:
    
    vit_image_size = 384
    
    class CustomViT(nn.Module):
        def __init__(self, model_arch, n_class, pretrained=False):
            super().__init__()
            self.model = timm.create_model(model_arch, pretrained=pretrained)
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, n_class)

        def forward(self, x):
            x = self.model(x)
            return x
        
    class TestDataset(Dataset):
        def __init__(self, df, transform=None):
            self.df = df
            self.file_names = df['image_path_id'].values
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            file_name = self.file_names[idx]
            im_bgr = cv2.imread(file_name)
            image = im_bgr[:, :, ::-1]
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            return image

    def get_tta_transforms():
        return Compose([CenterCrop(vit_image_size, vit_image_size, p=1.),
                Resize(vit_image_size, vit_image_size),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                ToTensorV2(p=1.0)], p=1.)

    def inference(models, test_loader, device):
        tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
        probs = []
        for i, (images) in tk0:
            avg_preds = []
            for model in models:
                images = images.to(device)
                model.to(device)
                model.eval()
                with torch.no_grad():
                    y_preds = model(images)
                avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probs.append(avg_preds)
        probs = np.concatenate(probs)
        return probs

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    predictions_vit = pd.DataFrame(columns={"image_id"})
    predictions_vit["image_id"] = submission_df["image_id"].values
    predictions_vit['image_path_id'] = image_path + predictions_vit['image_id'].astype(str)

    def load_cassava_vit(modelpath):
        _model = CustomViT('vit_base_patch16_384', 5, pretrained=False)
        _model.load_state_dict(torch.load(modelpath))
        _model.eval()
        return _model

    models = [load_cassava_vit(f) for f in used_models_pytorch["vit2020"]]

    test_dataset = TestDataset(predictions_vit, transform=get_tta_transforms())
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    predictions_raw_vit = inference(models, test_loader, device)

    predictions_vit['vit2020'] = [np.squeeze(p) for p in predictions_raw_vit]
    predictions_vit = predictions_vit.drop(["image_path_id"], axis=1)
    
    torch.cuda.empty_cache()
    try:
        for model in models:
            del(model)
    except:
        pass
    models = []
    gc.collect()

100%|██████████| 268/268 [06:41<00:00,  1.50s/it]


# Mobilenet V3 (CropNet)

There are multiple ways to include pretrained models from [TensorFlow Hub](https://www.tensorflow.org/hub/tutorials/cropnet_cassava), if internet has to be turned of during submission:

* Accessing and storing the .tar.gz file (see [this](https://xianbao-qian.medium.com/how-to-run-tf-hub-locally-without-internet-connection-4506b850a915) Medium post) 
<code>
!curl -LO https://storage.googleapis.com/tfhub-modules/google/cropnet/classifier/cassava_disease_V1/2.tar.gz
!mkdir cropnet_mobilenetv3
!tar -xf 2.tar.gz  --directory cropnet_mobilenetv3    
</code>
<br>

* Downloading and caching the weights using
<code>
os.environ["TFHUB_CACHE_DIR"] = "/kaggle/working"
hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=False)
</code>
<br>

You can find more [information on caching on the official tfhub website](https://www.tensorflow.org/hub/caching) and more information on the [pretrained CropNet model ](https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2). For the offline submissions we included these weights into a Kaggle Dataset bucket.

Remark: In the meantime, TFHub models can apparently be integrated directly into the TPU training via Kaggle. Check out the[ Kaggle TPU FAQs](https://www.kaggle.com/product-feedback/216256)



In [20]:
import tensorflow_hub as hub

def build_mobilenet3(img_size=(224,224), weights="cassava-leaf-disease-1st-place-models/cropnet_mobilenetv3/cropnet"):
    classifier = hub.KerasLayer(weights)
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
    hub.KerasLayer(classifier, trainable=False)])
    return model

# Keras Inference with TTA

For the included EfficientNets we used simple test time augmentations (Flip, Rotate, Transpose). To do this, we cropped 4 overlapping patches of size 512x512 from the .jpg images and applied 2 augmentations to each patch. We retain two additional center-cropped patches of the image to which no augmentations were applied. To get an overall prediction, we took the average of all these image tiles.

For the CropNet, we just center-cropped and resized the image. In addition, we distributed the unknown class evenly over the 5 leaf diseases.

In [21]:
def image_augmentations(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > 0.75:
        image = tf.image.transpose(image)
        
    if p_rotate > 0.75:
        image = tf.image.rot90(image, k = 3)
    elif p_rotate > 0.5:
        image = tf.image.rot90(image, k = 2)
    elif p_rotate > 0.25:
        image = tf.image.rot90(image, k = 1)

    image = tf.image.resize(image, size = IMAGE_SIZE)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

def read_preprocess_file(img_path, normalize=False):
    image = Image.open(img_path)
    if normalize:
        img_scaled = np.array(image)/ 255.0
    else:
        img_scaled = np.array(image)
    img_scaled = img_scaled.astype(np.float32)
    return (image.size[0], image.size[1]), img_scaled

def create_image_tiles(origin_dim, processed_img):
    crop_size = 512
    img_list = []
    # Cut image into 4 overlapping patches
    for x in [0, origin_dim[1] - crop_size]:
        for y in [0, origin_dim[0] - crop_size]:
            img_list.append(processed_img[x:x+crop_size , y:y+crop_size,:])
    # Keep one additional center cropped image 
    img_list.append(cv2.resize(processed_img[:, 100:700 ,:], dsize=(crop_size, crop_size)))
    return np.array(img_list)

def augment_tiles_light(tiles, ttas=2):
  # Copy central croped image to have same ratio to augmented images
  holdout = np.broadcast_to(tiles[-1,:,:,:],(ttas,) + tiles.shape[1:])
  augmented_batch = tf.map_fn(lambda x: image_augmentations(x), tf.concat(
      [tiles[:-1,:,:,:] for _ in range(ttas)], axis=0))
  return tf.concat([augmented_batch, holdout], axis=0)

def cut_crop_image(processed_img):
    image = tf.image.central_crop(processed_img, 0.8)
    image = tf.image.resize(image, (224, 224))
    return np.expand_dims(image, 0)

# CropNet class 6 (unknown) is distributed evenly over all 5 classes to match problem setting
def distribute_unknown(propabilities):
    return propabilities[:,:-1] + np.expand_dims(propabilities[:,-1]/5, 1)

def multi_predict_tfhublayer(img_path, modelinstance):
    img = cut_crop_image(read_preprocess_file(img_path, True)[1])
    yhat = modelinstance.predict(img)
    return np.mean(distribute_unknown(yhat), axis=0)

def multi_predict_keras(img_path, modelinstance, *args):
    augmented_batch = augment_tiles_light(create_image_tiles(
        *read_preprocess_file(img_path)))
    Yhat = modelinstance.predict(augmented_batch)
    return np.mean(Yhat, axis=0)

def predict_and_vote(image_list, modelinstances, onlykeras):
    predictions = [] 
    with tqdm(total=len(image_list)) as process_bar:       
      for img_path in image_list:
        process_bar.update(1)  
        Yhats = np.vstack([func(img_path, modelinstance) for func, modelinstance in modelinstances])
        if onlykeras:
            predictions.append(np.argmax(np.sum(Yhats, axis=0)))
        else:
            predictions.append(Yhats)    
    return predictions


inference_models = []

if "mobilenet" in used_models_keras:
    model_mobilenet = build_mobilenet3(weights=used_models_keras["mobilenet"])
    inference_models.append((multi_predict_tfhublayer, model_mobilenet))
    
if "efficientnetb4" in used_models_keras:
    model_efficientnetb4 =  keras.models.load_model(used_models_keras["efficientnetb4"], compile=False)
    inference_models.append((multi_predict_keras, model_efficientnetb4))
    
if "efficientnetb5" in used_models_keras:
    model_efficientnetb5 =  keras.models.load_model(used_models_keras["efficientnetb5"])
    inference_models.append((multi_predict_keras, model_efficientnetb5))

submission_df["label"] = predict_and_vote([image_path+id for id in submission_df["image_id"].values], inference_models, onlykeras)

2023-04-24 22:07:03.371238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 22:07:03.659033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 22:07:03.659822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 22:07:03.661147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

1/1 [==============================] - 3s 3s/step


2023-04-24 22:07:29.007704: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
  0%|          | 2/8558 [00:05<6:51:37,  2.89s/it]

1/1 [==============================] - 0s 44ms/step


  0%|          | 3/8558 [00:06<4:14:12,  1.78s/it]

...

100%|█████████▉| 8557/8558 [32:16<00:00,  4.43it/s]

1/1 [==============================] - 0s 35ms/step


100%|██████████| 8558/8558 [32:16<00:00,  4.40it/s]

1/1 [==============================] - 0s 35ms/step


100%|██████████| 8558/8558 [32:16<00:00,  4.42it/s]


In [22]:
tf.keras.backend.clear_session()

try:
    del inference_models[:]
except:
    pass

gc.collect()

91376

# Final Ensembling

Our winning submission just included CropNet, EfficientNet B4, ResNext50 and ViT and a mean approach. We took the mean of the class weights from the ResNext and ViT model and combined this combination with the MobileNet and the EfficientnetB4 in the second stage.

In [23]:
if len(list(used_models_keras.keys())) <= 1:
    submission_df.loc[:,list(used_models_keras)[0]] = submission_df["label"].explode()
else:
    tmp1 = (submission_df['label'].transform([lambda x:x[0]]).set_axis([list(used_models_keras.keys())[0]], axis=1, inplace=False))
    tmp2 = (submission_df['label'].transform([lambda x:x[1]]).set_axis([list(used_models_keras.keys())[1]], axis=1, inplace=False))
    submission_df = submission_df.merge(tmp1, right_index=True, left_index=True)
    submission_df = submission_df.merge(tmp2, right_index=True, left_index=True)
    
submission_df["label"] = 0

if "resnext" in used_models_pytorch:
    submission_df = submission_df.merge(predictions_resnext, on="image_id")
    
if "efficientnetb3" in used_models_pytorch:
    submission_df = submission_df.merge(predictions_cutmix, on="image_id")
    
if "vit2020" in used_models_pytorch:
    submission_df = submission_df.merge(predictions_vit, on="image_id")
    
if "vit2019" in used_models_pytorch:
    submission_df = submission_df.merge(predictions_vit2019, on="image_id")

In [24]:
if stacked_mean:
    submission_df["stage_1"] = submission_df.apply(lambda row: [np.mean(e) for e in zip(row["vit2020"], row["resnext"])], axis=1)
    submission_df["label"] = submission_df.apply(lambda row: np.argmax(
        [np.sum(e) for e in zip(row["mobilenet"],row["stage_1"], row["efficientnetb4"])]), axis=1)        
else:
    submission_df["label"] = submission_df.apply(lambda row: np.argmax(
        [np.sum(e) for e in zip(*[row[m] for m in list(used_models_pytorch.keys())+list(used_models_keras.keys())])]), axis=1)

In [25]:
submission_df.head(1)

,image_id,label,mobilenet,efficientnetb4,resnext,vit2020,stage_1
0,4257577206.jpg,0,"[0.9383143, 0.0019460203, 0.00019338421, 0.000...","[0.45157585, 0.18849367, 0.058139514, 0.047221...","[0.67888737, 0.09235903, 0.009589413, 0.006527...","[0.6695668, 0.005161765, 0.0005797537, 0.00033...","[0.6742271, 0.048760395, 0.0050845835, 0.00342..."


In [26]:
submission_df[["image_id","label"]].to_csv("submission.csv", index=False)
!head submission.csv

image_id,label
4257577206.jpg,0
3158652079.jpg,0
545843686.jpg,3
2828087857.jpg,3
3434116989.jpg,4
2581730862.jpg,4
728221316.jpg,3
2016298412.jpg,3
1072929806.jpg,3


In [27]:
submission_df.to_csv("all.csv")

In [32]:
truth_label = ground_truth_df['label'].to_numpy(dtype=int)
predict_label = submission_df['label'].to_numpy(dtype=int)

In [33]:
def calc_f1(pred, label, num_classes, average):
    return multiclass_f1_score(torch.from_numpy(pred.astype(int)), torch.from_numpy(label.astype(int)), num_classes=num_classes, average=average)

In [34]:
calc_f1(predict_label, truth_label, 5, None)

tensor([0.8030, 0.8850, 0.8714, 0.9735, 0.8304])

In [37]:
calc_f1(predict_label, truth_label, 5, 'micro')

tensor(0.9284)

In [38]:
calc_f1(predict_label, truth_label, 5, 'macro')

tensor(0.8727)